## SET UP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/LauraMJanssen/espcn.git
%cd espcn

In [ ]:
!pip install keras-tqdm
!sudo pip install scikit-video

## TRAIN

For transfer learning: \
remove '#' in line 303 in 'espcn/train.py'

In [ ]:
!python3 train.py --train /content/drive/MyDrive/IVUSImages/HR_Train \
                  --validation /content/drive/MyDrive/IVUSImages/HR_Valid \
                  --test /content/drive/MyDrive/IVUSImages/HR_Valid \
                  --scale 4 \
                  --stage default \
                  --steps_per_epoch 677 \
                  --epochs 20 

In [ ]:
!zip -r /content/ESRCN_model.zip /content/ESPCN-Keras/model/ESPCN_4X.h5

In [ ]:
from google.colab import files
files.download("/content/ESRCN_model.zip")

## TEST

(1) Create necessary directories

In [12]:
!mkdir /content/results

In [ ]:
!mkdir /content/espcn/logs

(2) Define quality metric functions

In [14]:
from skimage.metrics import structural_similarity as ssim

# define a function for peak signal-to-noise ratio (PSNR)
def psnr(target, ref):
         
    # assume RGB image
    target_data = target.astype(float)
    ref_data = ref.astype(float)

    diff = ref_data - target_data
    diff = diff.flatten('C')

    rmse = math.sqrt(np.mean(diff ** 2.))

    return 20 * math.log10(255. / rmse)

# define function for mean squared error (MSE)
def mse(target, ref):
    # the MSE between the two images is the sum of the squared difference
    err = np.sum((target.astype('float') - ref.astype('float')) ** 2)
    err /= float(target.shape[0] * target.shape[1])
    
    return err

# define function that combines all three image quality metrics
def compare_images(target, ref):
    scores = []
    scores.append(psnr(target, ref))
    scores.append(mse(target, ref))
    scores.append(ssim(target, ref, multichannel =True))
    
    return scores

(3) Execute model on all images

In [ ]:
!python libs/espcn.py

(4) Print results \
(make sure to adapt ref and degraded parts according to image read above)

In [ ]:
import math
import numpy as np
import cv2

HR = cv2.imread('/content/drive/MyDrive/IVUSImages/HR_Valid/001.png')
LR = cv2.imread('/content/drive/MyDrive/IVUSImages/LR_Scale4_Valid/001.png')
SR = cv2.imread('/content/results/001.png')
  
# image quality calculations
scores = []
h, w, _ = LR.shape
LR2 = cv2.resize(LR, (4*w, 4*h))
scores.append(compare_images(LR2, HR))

scores.append(compare_images(SR, HR))

  
# print all scores for all images
print('Degraded Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'
        .format(scores[0][0], scores[0][1], scores[0][2]))
print('Reconstructed Image: \nPSNR: {}\nMSE: {}\nSSIM: {}\n'
        .format(scores[1][0], scores[1][1], scores[1][2]))